In [2]:
import streamlit as st
import json
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.chat_models import ChatOpenAI
import vosk
import pyttsx3
import sounddevice as sd
import numpy as np
import wave
import re
import uuid

In [3]:
test_sentence="Transfer $65 to my niece for the dance lessons"

In [4]:
model = BertForSequenceClassification.from_pretrained("C:/Users/hp/Downloads/lm/Model_bert_ds_b1")
tokenizer = BertTokenizer.from_pretrained("C:/Users/hp/Downloads/lm/Model_bert_ds_b1")

inputs = tokenizer(test_sentence, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

predicted_labels = torch.argmax(outputs.logits, dim=1)
int(predicted_labels)

2

In [5]:
json_file_path = "C:/Users/hp/Downloads/lm/database.json"

In [6]:

def get_account_details(account_number):
    # Replace this with your logic to fetch account details from your data source
    # For example, you can use a JSON file to store account details
    with open(json_file_path, 'r') as f:
        data = json.load(f)

    for account in data['accounts']:
        if account['acc_no'] == int(account_number):
            return {
                "name": account['name'],
                "acc_no": account['acc_no'],
                "balance": account['balance']
            }

    return None

In [12]:
get_account_details("832857").get('balance', 0)

525.0

In [13]:
get_account_details("832857").get('dues', 0)

0

In [14]:
get_account_details("832857")

{'name': 'Chiranjeevi', 'acc_no': 832857, 'balance': 525.0}

In [10]:
def update_account_balance(account_number, amount):
    # Load account data from JSON file
    with open(json_file_path, "r") as file:
        account_data = json.load(file)

    # Find the account in the data
    for account in account_data["accounts"]:
        if account["acc_no"] == account_number:
            # Update the balance
            account["balance"] += amount
            break

    # Save the updated data back to the JSON file
    with open(json_file_path, "w") as file:
        json.dump(account_data, file, indent=2)

def transfer_money(sender_acc_no, receiver_acc_no, amount):
    data = read_data_from_json(json_file_path)
    sender_account = None
    receiver_account = None

    # Find sender and receiver accounts
    for account in data['accounts']:
        if account['acc_no'] == int(sender_acc_no):
            sender_account = account
        elif account['acc_no'] == int(receiver_acc_no):
            receiver_account = account

    # Check if sender and receiver accounts exist
    if sender_account is None or receiver_account is None:
        return False, "Sender or receiver account not found."

    # Check if sender has sufficient balance
    if sender_account['balance'] < amount:
        return False, "Insufficient balance."

    # Deduct amount from sender's balance
    sender_account['balance'] -= amount

    # Add amount to receiver's balance
    receiver_account['balance'] += amount

    # Update JSON data
    write_data_to_json(json_file_path, data)

    return True, "Transfer successful. You have transferred " + str(amount) + " to " + str(receiver_acc_no) + "."

def read_data_from_json(filename):
    with open(filename, 'r') as f:
        data = json.load(f)
    return data

def write_data_to_json(filename, data):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4)

In [15]:
transfer_money("832857","832856", 25.0)

(True, 'Transfer successful. You have transferred 25.0 to 832856.')